# functions: write_result

[write_result](../api/functions.rst#nornir_utils.plugins.functions.write_result) is a function that writes result and returns diff between old file and new file:

In [1]:
from nornir import InitNornir

def dummy_task(task):
    return "hi!!!"

nr = InitNornir(
    inventory={
        "plugin":"YAMLInventory",
        "options": {
            "host_file": "data/hosts.yaml",
            "group_file": "data/groups.yaml",
            "defaults_file": "data/defaults.yaml",
        }
    }
)

result = nr.run(task=dummy_task)

Now we could write the result easily with the `write_result` function:

In [2]:
from nornir_utils.plugins.functions import write_result

# write_result returns diff between the previous and current file state
write_result(result, filename="out_files/write_result/result.txt")

'--- out_files/write_result/result.txt\n\n+++ new\n\n@@ -0,0 +1,19 @@\n\n+---- dev1.group_1: dummy_task ** changed : False ------------------------------- INFO\n+\n+hi!!!\n+\n+---- dev2.group_1: dummy_task ** changed : False ------------------------------- INFO\n+\n+hi!!!\n+\n+---- dev3.group_2: dummy_task ** changed : False ------------------------------- INFO\n+\n+hi!!!\n+\n+---- dev4.group_2: dummy_task ** changed : False ------------------------------- INFO\n+\n+hi!!!\n+\n+---- dev5.no_group: dummy_task ** changed : False ------------------------------ INFO\n+\n+hi!!!'

In [3]:
with open("out_files/write_result/result.txt") as f:
    print(f.read())

---- dev1.group_1: dummy_task ** changed : False ------------------------------- INFO

hi!!!

---- dev2.group_1: dummy_task ** changed : False ------------------------------- INFO

hi!!!

---- dev3.group_2: dummy_task ** changed : False ------------------------------- INFO

hi!!!

---- dev4.group_2: dummy_task ** changed : False ------------------------------- INFO

hi!!!

---- dev5.no_group: dummy_task ** changed : False ------------------------------ INFO

hi!!!


`filename` here is path to file, you want to write the result. `write_result` creates directories from the path with filename, if it's necessary.

## Diff

As you can see, `write_result` returns diff between the previous and current file state:

In [4]:
def smart_task(task):
    return "hihi!!!"

result = nr.run(task=smart_task)

print(write_result(result, filename="out_files/write_result/result.txt"))

--- out_files/write_result/result.txt

+++ new

@@ -0,0 +1,19 @@

+---- dev1.group_1: smart_task ** changed : False ------------------------------- INFO
+
+hihi!!!
+
+---- dev2.group_1: smart_task ** changed : False ------------------------------- INFO
+
+hihi!!!
+
+---- dev3.group_2: smart_task ** changed : False ------------------------------- INFO
+
+hihi!!!
+
+---- dev4.group_2: smart_task ** changed : False ------------------------------- INFO
+
+hihi!!!
+
+---- dev5.no_group: smart_task ** changed : False ------------------------------ INFO
+
+hihi!!!


In [5]:
with open("out_files/write_result/result.txt") as f:
    print(f.read())

---- dev1.group_1: smart_task ** changed : False ------------------------------- INFO

hihi!!!

---- dev2.group_1: smart_task ** changed : False ------------------------------- INFO

hihi!!!

---- dev3.group_2: smart_task ** changed : False ------------------------------- INFO

hihi!!!

---- dev4.group_2: smart_task ** changed : False ------------------------------- INFO

hihi!!!

---- dev5.no_group: smart_task ** changed : False ------------------------------ INFO

hihi!!!


## Writing specific data

If the task returns different information, you can also select which ones to write. For instance:

In [6]:
from nornir.core.task import Result

def task_with_data(task):
    return Result(host=task.host, result="Spaghetti monster", sw_char="Jar Jar Binks", food="hawaiian pizza", OS="windows")


fav_result = nr.run(task=task_with_data)

# write only sw_char, food vars and write diff to diff variable 
diff = write_result(fav_result, filename="out_files/write_result/result.txt", vars=["sw_char", "food"])

In [7]:
with open("out_files/write_result/result.txt") as f:
    print(f.read())

---- dev1.group_1: task_with_data ** changed : False --------------------------- INFO

Jar Jar Binks

hawaiian pizza

---- dev2.group_1: task_with_data ** changed : False --------------------------- INFO

Jar Jar Binks

hawaiian pizza

---- dev3.group_2: task_with_data ** changed : False --------------------------- INFO

Jar Jar Binks

hawaiian pizza

---- dev4.group_2: task_with_data ** changed : False --------------------------- INFO

Jar Jar Binks

hawaiian pizza

---- dev5.no_group: task_with_data ** changed : False -------------------------- INFO

Jar Jar Binks

hawaiian pizza


## Severity

You can also ask `write_result` to write the results only if the severity is equal or superior to the one specified. This is particularly useful if your script is very large and you don't care about intermediate results. For instance:

In [8]:
import logging

def complex_task(task):
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    return "I did a lot of things!!!"

complex_result = nr.run(task=complex_task)

`write_result` will only write `INFO` severity and above by default so if we write the result we should only see the result of the parent task:

In [9]:
diff = write_result(complex_result, filename="out_files/write_result/result.txt")

In [10]:
with open("out_files/write_result/result.txt") as f:
    print(f.read())

---- dev1.group_1: complex_task ** changed : False ----------------------------- INFO

I did a lot of things!!!

---- dev2.group_1: complex_task ** changed : False ----------------------------- INFO

I did a lot of things!!!

---- dev3.group_2: complex_task ** changed : False ----------------------------- INFO

I did a lot of things!!!

---- dev4.group_2: complex_task ** changed : False ----------------------------- INFO

I did a lot of things!!!

---- dev5.no_group: complex_task ** changed : False ---------------------------- INFO

I did a lot of things!!!


If you wanted to write all the tasks for debugging purposes you could specify it with:

In [11]:
diff = write_result(complex_result, filename="out_files/write_result/result.txt", severity_level=logging.DEBUG)

In [12]:
with open("out_files/write_result/result.txt") as f:
    print(f.read())

---- dev1.group_1: complex_task ** changed : False ----------------------------- INFO

I did a lot of things!!!

---- dev1.group_1: dummy_task ** changed : False ------------------------------- DEBUG

hi!!!

---- dev1.group_1: dummy_task ** changed : False ------------------------------- DEBUG

hi!!!

---- dev1.group_1: dummy_task ** changed : False ------------------------------- DEBUG

hi!!!

---- dev1.group_1: dummy_task ** changed : False ------------------------------- DEBUG

hi!!!

---- dev1.group_1: dummy_task ** changed : False ------------------------------- DEBUG

hi!!!

---- dev2.group_1: complex_task ** changed : False ----------------------------- INFO

I did a lot of things!!!

---- dev2.group_1: dummy_task ** changed : False ------------------------------- DEBUG

hi!!!

---- dev2.group_1: dummy_task ** changed : False ------------------------------- DEBUG

hi!!!

---- dev2.group_1: dummy_task ** changed : False ------------------------------- DEBUG

hi!!!

---- dev2.grou

## Write host

`write_host` argument can help you, if you don't need to write hostnames to file:

In [13]:
diff = write_result(result, filename="out_files/write_result/result.txt", write_host=False)

In [14]:
with open("out_files/write_result/result.txt") as f:
    print(f.read())

---- smart_task ** changed : False --------------------------------------------- INFO

hihi!!!

---- smart_task ** changed : False --------------------------------------------- INFO

hihi!!!

---- smart_task ** changed : False --------------------------------------------- INFO

hihi!!!

---- smart_task ** changed : False --------------------------------------------- INFO

hihi!!!

---- smart_task ** changed : False --------------------------------------------- INFO

hihi!!!


## Append

`append` argument allows you to use `a+` or `w+` (default mode) writing mode. As instance:

In [15]:
diff = write_result(fav_result, filename="out_files/write_result/result.txt", append=True)

In [16]:
with open("out_files/write_result/result.txt") as f:
    print(f.read())

---- smart_task ** changed : False --------------------------------------------- INFO

hihi!!!

---- smart_task ** changed : False --------------------------------------------- INFO

hihi!!!

---- smart_task ** changed : False --------------------------------------------- INFO

hihi!!!

---- smart_task ** changed : False --------------------------------------------- INFO

hihi!!!

---- smart_task ** changed : False --------------------------------------------- INFO

hihi!!!

---- dev1.group_1: task_with_data ** changed : False --------------------------- INFO

Spaghetti monster

---- dev2.group_1: task_with_data ** changed : False --------------------------- INFO

Spaghetti monster

---- dev3.group_2: task_with_data ** changed : False --------------------------- INFO

Spaghetti monster

---- dev4.group_2: task_with_data ** changed : False --------------------------- INFO

Spaghetti monster

---- dev5.no_group: task_with_data ** changed : False -------------------------- INFO

Spaghetti

# No errors

Don't write errors to file with `no_errors` argument. As instance:

In [17]:
def task_with_exception(task):
    return Result(host=task.host, result="Something went wrong", exception=Exception())


def task_without_exception(task):
    return "All is fine"

def task(task):
    task.run(task=task_with_exception)
    task.run(task=task_without_exception)
    return "What's happening?"

    
result_with_exception = nr.run(task=task)

diff = write_result(result_with_exception, filename="out_files/write_result/result.txt")

In [18]:
with open("out_files/write_result/result.txt") as f:
    print(f.read())

---- dev1.group_1: task ** changed : False ------------------------------------- INFO

What's happening?

---- dev1.group_1: task_with_exception ** changed : False ---------------------- INFO

Something went wrong

---- dev1.group_1: task_without_exception ** changed : False ------------------- INFO

All is fine

---- dev2.group_1: task ** changed : False ------------------------------------- INFO

What's happening?

---- dev2.group_1: task_with_exception ** changed : False ---------------------- INFO

Something went wrong

---- dev2.group_1: task_without_exception ** changed : False ------------------- INFO

All is fine

---- dev3.group_2: task ** changed : False ------------------------------------- INFO

What's happening?

---- dev3.group_2: task_with_exception ** changed : False ---------------------- INFO

Something went wrong

---- dev3.group_2: task_without_exception ** changed : False ------------------- INFO

All is fine

---- dev4.group_2: task ** changed : False ------------

In [19]:
diff = write_result(result_with_exception, filename="out_files/write_result/result.txt", no_errors=True)

In [20]:
with open("out_files/write_result/result.txt") as f:
    print(f.read())

---- dev1.group_1: task ** changed : False ------------------------------------- INFO

What's happening?

---- dev1.group_1: task_without_exception ** changed : False ------------------- INFO

All is fine

---- dev2.group_1: task ** changed : False ------------------------------------- INFO

What's happening?

---- dev2.group_1: task_without_exception ** changed : False ------------------- INFO

All is fine

---- dev3.group_2: task ** changed : False ------------------------------------- INFO

What's happening?

---- dev3.group_2: task_without_exception ** changed : False ------------------- INFO

All is fine

---- dev4.group_2: task ** changed : False ------------------------------------- INFO

What's happening?

---- dev4.group_2: task_without_exception ** changed : False ------------------- INFO

All is fine

---- dev5.no_group: task ** changed : False ------------------------------------ INFO

What's happening?

---- dev5.no_group: task_without_exception ** changed : False --------